In [1]:
%load_ext autoreload
import os
os.chdir(os.path.abspath(os.curdir).replace('/notebooks',''))

In [28]:
%autoreload 2
from src.database import ElasticTransformers

In [3]:
from sentence_transformers import SentenceTransformer

bert_embedder = SentenceTransformer('bert-base-nli-mean-tokens')

In [16]:
def embed_wrapper(ls):
    """
    Helper function which simplifies the embedding call and helps lading data into elastic easier
    """
    results=bert_embedder.encode(ls)
    results = [r.to_list() for r in results]
    return results


In [33]:
et=ElasticTransformers(index_name='et-test')
et.ping()

True

# Size matters
Below we search for `security thead` in the tiny (1k records) vs the large (1.1mn records) index. 
 - keyword matching takes 12ms on tiny and 45ms (4x) in the large index 
 - dense (embedding) search takse 400ms on tiny and 8000ms (20x) on the large index

In [45]:
%%time
_ = et.search('security thread','headline_text',index_name='et-tiny',type='dense',embedder=embed_wrapper, size=100)

2020-08-31 12:08:24,711 - src.logger - DEBUG - search:251 - Search DENSE security thread in et-tiny.headline_text returned 100 results of 100 requested
DEBUG:src.logger:Search DENSE security thread in et-tiny.headline_text returned 100 results of 100 requested


CPU times: user 179 ms, sys: 11.9 ms, total: 191 ms
Wall time: 431 ms


In [46]:
%%time
_ = et.search('security thread','headline_text',index_name='et-tiny',type='match',embedder=embed_wrapper, size=100)

2020-08-31 12:08:50,569 - src.logger - DEBUG - search:251 - Search MATCH security thread in et-tiny.headline_text returned 5 results of 100 requested
DEBUG:src.logger:Search MATCH security thread in et-tiny.headline_text returned 5 results of 100 requested


CPU times: user 3.37 ms, sys: 1.03 ms, total: 4.4 ms
Wall time: 12.7 ms


In [42]:
%%time
_ = et.search('security thread','headline_text',type='dense',embedder=embed_wrapper, size=100)


2020-08-31 11:39:17,979 - src.logger - DEBUG - search:251 - Search DENSE security thread in None.headline_text returned 100 results of 100 requested
DEBUG:src.logger:Search DENSE security thread in None.headline_text returned 100 results of 100 requested


CPU times: user 178 ms, sys: 12.8 ms, total: 191 ms
Wall time: 8.31 s


In [43]:
%%time
_ = et.search('security thread','headline_text',type='dense',embedder=embed_wrapper, size=5)


2020-08-31 11:39:27,313 - src.logger - DEBUG - search:251 - Search DENSE security thread in None.headline_text returned 5 results of 5 requested
DEBUG:src.logger:Search DENSE security thread in None.headline_text returned 5 results of 5 requested


CPU times: user 177 ms, sys: 12.1 ms, total: 189 ms
Wall time: 8.22 s


In [44]:
%%time
_ = et.search('news','headline_text',type='dense',embedder=embed_wrapper, size=5)


2020-08-31 11:39:54,702 - src.logger - DEBUG - search:251 - Search DENSE news in None.headline_text returned 5 results of 5 requested
DEBUG:src.logger:Search DENSE news in None.headline_text returned 5 results of 5 requested


CPU times: user 186 ms, sys: 14.2 ms, total: 200 ms
Wall time: 8.34 s


In [44]:
%%time
_ = et.search('news','headline_text',type='dense',embedder=embed_wrapper, size=5)


2020-08-31 11:39:54,702 - src.logger - DEBUG - search:251 - Search DENSE news in None.headline_text returned 5 results of 5 requested
DEBUG:src.logger:Search DENSE news in None.headline_text returned 5 results of 5 requested


CPU times: user 186 ms, sys: 14.2 ms, total: 200 ms
Wall time: 8.34 s


In [41]:
%%time
et.search('virus thread','headline_text',type='match',embedder=embed_wrapper, size = 100)

2020-08-31 11:38:56,249 - src.logger - DEBUG - search:251 - Search MATCH virus thread in None.headline_text returned 100 results of 100 requested
DEBUG:src.logger:Search MATCH virus thread in None.headline_text returned 100 results of 100 requested


CPU times: user 12.9 ms, sys: 2.25 ms, total: 15.2 ms
Wall time: 44.9 ms


,_score,headline_text,headline_text_embedding
0,12.561751,open thread june 13 2014,"[-0.0028050158, 0.1959735453, 0.8845489621, 0...."
1,12.159321,open thread friday june 20,"[-0.1166632324, -0.4709567428, 1.6022088528, 0..."
2,11.530302,open thread: may 23 2014,"[-0.297937274, 0.0036600307, 1.2500419617, 0.3..."
3,11.530302,open thread: may 30 2014,"[-0.3281069994, -0.055470407, 1.5308458805, 0...."
4,11.368732,temers presidency hanging by a thread,"[0.6944692135, 0.0708749965, 1.3958420753, 0.8..."
...,...,...,...
95,8.425070,hendra virus vaccine update,"[0.3527559638, -0.3274123073, 0.9984663725, -0..."
96,8.352560,hendra virus flying foxes,"[0.1032108814, 0.8079137802, -0.3388480842, -0..."
97,8.352560,hendra virus fact box,"[0.4632798135, 0.081611529, 0.5122417808, 0.28..."
98,8.352560,virus transport plan scrapped,"[0.176388219, 0.4736163318, 0.8430795074, 0.34..."


In [19]:
import pandas as pd
df=pd.read_csv('data/abcnews-date-text.csv')

In [20]:
df.shape

(1186018, 2)